In [44]:
import os
import numpy as np
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

countt=0
fracture = []
nonfrac = []
for dirname, _, filenames in os.walk('/kaggle/input/fracatlas/FracAtlas/images/Fractured'):
    for filename in filenames:
        img_path  = os.path.join(dirname,filename)
        img = image.load_img(img_path, target_size=(224, 224,3))
        x_img = image.img_to_array(img)
        x_img = np.expand_dims(x_img, axis=0)
        x_imp = preprocess_input(x_img)
        fracture.append(x_img[0])
        countt+=1
        if countt==50:
            break

countt = 0
for dirname, _, filenames in os.walk('/kaggle/input/fracatlas/FracAtlas/images/Non_fractured'):
    for filename in filenames:
        img_path  = os.path.join(dirname,filename)
        try:
            img = image.load_img(img_path, target_size=(224, 224,3))
            x_img = image.img_to_array(img)
            x_img = np.expand_dims(x_img, axis=0)
            x_imp = preprocess_input(x_img)
            if x_img is not None:
                nonfrac.append(x_img[0])
                countt+=1
                if countt==50:
                    break
        except:
            print(f"Error processing image {img_path}")

Error processing image /kaggle/input/fracatlas/FracAtlas/images/Non_fractured/IMG0004347.jpg
Error processing image /kaggle/input/fracatlas/FracAtlas/images/Non_fractured/IMG0004297.jpg
Error processing image /kaggle/input/fracatlas/FracAtlas/images/Non_fractured/IMG0004148.jpg


In [45]:
fracture[0].shape

(224, 224, 3)

In [46]:
fracture_array = np.array(fracture)
nonfrac_array = np.array(nonfrac)

combined_array = np.concatenate((fracture_array, nonfrac_array), axis=0)

print(combined_array.shape)


(100, 224, 224, 3)


In [47]:
sz1 = 50
sz2 = 50

array1 = np.array([1, 0])
array2 = np.array([0, 1])

repeated_array1 = np.tile(array1, (sz1, 1))
repeated_array2 = np.tile(array2, (sz2, 1))

result_array = np.concatenate((repeated_array1, repeated_array2), axis=0)

print(result_array.shape)
#print(result_array)

(100, 2)


In [48]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(combined_array, result_array, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


X_train shape: (80, 224, 224, 3)
X_test shape: (20, 224, 224, 3)
y_train shape: (80, 2)
y_test shape: (20, 2)


In [55]:
# look how classification of images in dataset
fr = 0
nf = 0
for y in y_train:
    if y[0]==1:
        fr += 1
    elif y[0]==0:
        nf += 1
        
print(f"fractured images = {fr}\nnon fractured images = {nf}\ntotal images = {fr+nf}")

fractured images = 38
non fractured images = 42
total images = 80


In [49]:
from keras.applications.vgg16 import VGG16
from keras.models import Model

# Load the pre-trained VGG model without the top (fully connected) layers
base_model = VGG16(weights=None, include_top=False, input_shape=(224, 224, 3))

# Create a new model with the VGG base model's input and output
vgg = Model(inputs=base_model.input, outputs=base_model.output)

# Load weights from the H5 file
vgg.load_weights('/kaggle/input/vgg16-weights-tf-keras/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5')

In [50]:
from keras.layers import Dense, GlobalAveragePooling2D

x = vgg.output
x = GlobalAveragePooling2D()(x)
x = Dense(2, activation='sigmoid')(x)

model = Model(inputs=vgg.input, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.summary()

Model: "functional_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_3      │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │         1,026 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,715,714 (56.14 MB)

 Trainable params: 14,715,714 (56.14 MB)

 Non-trainable params: 0 (0.00 B)

In [51]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 87s 26s/step - accuracy: 0.4672 - loss: 152364.7188 - val_accuracy: 0.4000 - val_loss: 1.3502
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 141s 26s/step - accuracy: 0.5031 - loss: 0.9071 - val_accuracy: 0.4000 - val_loss: 17.7206
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 142s 26s/step - accuracy: 0.5437 - loss: 7.9916 - val_accuracy: 0.6000 - val_loss: 0.6833
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 142s 26s/step - accuracy: 0.5031 - loss: 0.6984 - val_accuracy: 0.6000 - val_loss: 0.6869
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 141s 26s/step - accuracy: 0.4680 - loss: 0.6962 - val_accuracy: 0.6000 - val_loss: 0.6888
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 142s 26s/step - accuracy: 0.4875 - loss: 0.6944 - val_accuracy: 0.6000 - val_loss: 0.6912
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 142s 26s/step - accuracy: 0.4563 - loss: 0.6944 - val_accuracy: 0.6000 - val_loss: 0.6919
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 142s 26s/step - accuracy: 0.4523 - loss: 0.6936 - val_accuracy: 0.6000 - val_

In [52]:
loss, accuracy = model.evaluate(X_test, y_test)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - accuracy: 0.6000 - loss: 0.6934
Test Loss: 0.693351686000824
Test Accuracy: 0.6000000238418579


In [56]:
# Lets look at  fractured image
img_path = '/kaggle/input/fracatlas/FracAtlas/images/Fractured/IMG0000019.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x_img = image.img_to_array(img)
x_img = np.expand_dims(x_img, axis=0)
x_imp = preprocess_input(x_img)

features = model.predict(x_img)

features

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 480ms/step


array([[0.49853635, 0.4977944 ]], dtype=float32)

In [58]:
sum(features[0])

0.996330738067627

As 0.4985365 > 0.4977944 hence we say that label at index 0 which correponds to fracture has higher probability here.